In [ ]:
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-en-vi-base")
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-base")
model.to(device)

src = "I'm feeling under the weather today"
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
print(tokenized_text)
model.eval()
summary_ids = model.generate(
                    tokenized_text,
                    max_length=128, 
                    num_beams=5,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                )
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(model))

## Reading data file, saving result

In [ ]:
with open('tst2013.en','r') as f:
    eng_txt = f.read().splitlines()
print(eng_txt)

In [ ]:
with open('tst2013.vi','r',encoding = 'utf8') as fv:
    vie_txt = fv.read().splitlines()
print(vie_txt)

In [ ]:
# src = "Good weather today ."
src = eng_txt[3]
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
model.eval()
summary_ids = model.generate(
                    tokenized_text,
                    max_length=128, 
                    num_beams=5,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                )
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

In [ ]:
## Legacy original prediction and save code
# from tqdm import tqdm
# with open('tst2013_original.pred','w',encoding = 'utf8') as fp:
#     for t in tqdm(eng_txt[:5]):
#         src = t
#         tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
#         model.eval()
#         summary_ids = model.generate(
#                             tokenized_text,
#                             max_length=128, 
#                             num_beams=5,
#                             repetition_penalty=2.5, 
#                             length_penalty=1.0, 
#                             early_stopping=True
#                         )
#         output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#         fp.write(output + '\n')

In [ ]:
from tqdm import tqdm
results = []
for t in tqdm(eng_txt[:]):
    src = t
    tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
    model.eval()
    summary_ids = model.generate(
                        tokenized_text,
                        max_length=128, 
                        num_beams=5,
                        repetition_penalty=2.5, 
                        length_penalty=1.0, 
                        early_stopping=True
                    )
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    results.append(output)


In [ ]:
with open('tst2013.pred','w',encoding = 'utf8') as fp:
    for result in results:
        fp.write(result + '\n')

In [ ]:
with open("tst2013.pred",'r',encoding = 'utf8') as fp:
    pred_txt = fp.read().splitlines()

## Saving model

In [ ]:
save_file_path = ('models/model.pth')  #save with fp32
torch.save(model.state_dict(), save_file_path)

In [3]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer,AutoConfig

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-base")

# Initialize the model with the architecture

config = AutoConfig.from_pretrained("NlpHUST/t5-en-vi-base")
new_model = T5ForConditionalGeneration(config=config)

# Load the weights from the .pth file
weights_path = "models/model.pth"
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))  # Load the weights onto CPU

# Load the state_dict onto the model
new_model.load_state_dict(state_dict)

# Move the model to the desired device
device = "cpu"  # Change this to "cuda" if you want to use GPU
new_model.to(device)

# Iterate through the parameters and print their datatype
param_dtype_list = []
for name, param in new_model.named_parameters():
    # print(f"Parameter '{name}' has datatype: {param.dtype}")
    if param.dtype not in param_dtype_list:
        param_dtype_list.append(param.dtype)
print(param_dtype_list)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[torch.float32]


In [ ]:
# Test loaded model
# src = "I'm feeling under the weather today"
# tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
# print(tokenized_text)
# new_model.eval()
# summary_ids = new_model.generate(
#                     tokenized_text,
#                     max_length=128, 
#                     num_beams=5,
#                     repetition_penalty=2.5, 
#                     length_penalty=1.0, 
#                     early_stopping=True
#                 )
# output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# print(output)

In [4]:
import torch.quantization
	
quantized_model = torch.quantization.quantize_dynamic(new_model, {torch.nn.Linear}, dtype=torch.qint8)

In [7]:
#Test quantized model
quantized_model.to(device)
device = "cuda"
src = "I'm feeling under the weather today"
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
print(tokenized_text)
quantized_model.eval()
summary_ids = quantized_model.generate(
                    tokenized_text,
                    max_length=128, 
                    num_beams=5,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                )
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

tensor([[  336,   277,   282, 28478,  1711,   287, 40864,  7883,     1]],
       device='cuda:0')


NotImplementedError: Could not run 'quantized::linear_dynamic' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_dynamic' is only available for these backends: [CPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at /opt/conda/conda-bld/pytorch_1666643016022/work/aten/src/ATen/native/quantized/cpu/qlinear_dynamic.cpp:656 [kernel]
B

In [ ]:

# Inference from quantized model
device = "cuda"
quantized_model.to(device)
from tqdm import tqdm
results = []
for t in tqdm(eng_txt[:]):
    src = t
    tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)
    quantized_model.eval()
    summary_ids = quantized_model.generate(
                        tokenized_text,
                        max_length=128, 
                        num_beams=5,
                        repetition_penalty=2.5, 
                        length_penalty=1.0, 
                        early_stopping=True
                    )
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    results.append(output)

In [ ]:
save_file_path = ('quantized_models/model.pth')  #save with fp32
torch.save(quantized_model.state_dict(), save_file_path)

In [ ]:
# # Saving model config (for architecture)
# from transformers import AutoConfig
# import json

# # Load the model's configuration
# config = AutoConfig.from_pretrained("NlpHUST/t5-en-vi-base")

# # Convert the configuration to a dictionary
# config_dict = config.to_dict()

# # Define the path to save the JSON file
# json_path = "model_config.json"

# # Write the configuration dictionary to a JSON file
# with open(json_path, 'w') as json_file:
#     json.dump(config_dict, json_file, indent=4)

## BLEU Score


In [ ]:
def split_string(string):
    return string.split()
predictions = list(map(split_string, pred_txt))
references = list(map(split_string,vie_txt[:]))
print(predictions)
print(references)

In [ ]:
import nltk
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

ref_txt = vie_txt[:]
ref_len=len(ref_txt)
pred_len=len(pred_txt)
chencherry = SmoothingFunction()
# assert ref_len==pred_len

bleu=0
for i in tqdm(range(ref_len)): 
    # Only consider Unigram BLEU score here (n = 1)
    sent_bleu = sentence_bleu([ref_txt[i].strip().split()],pred_txt[i].strip().split(),smoothing_function = chencherry.method3,weights = (1,0,0,0))
    bleu+=sent_bleu
    
print('\nAverage Unigram BLEU Score:',bleu/float(ref_len))